In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import random
import re

In [ ]:
# Ruta actualizada del archivo
file_path ='ruta del archivo'

# Intentar leer el archivo y separar columnas
try:
    data = pd.read_csv(file_path, encoding='utf-8', delimiter=';')  # Probar primero con utf-8 y delimitador ;
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='latin1', delimiter=';') 
except FileNotFoundError:
    print("El archivo no se encuentra en la ruta especificada. Verifica nuevamente.")

# Mostrar las primeras filas para verificar
print(data.head())

# Verificar si las columnas están separadas correctamente
if 'text' not in data.columns:
    print("Error: La columna 'text' no existe. Verifique el delimitador del archivo CSV.")
else:
    print("Archivo cargado correctamente con las columnas esperadas.")

In [ ]:
# Crear una función para limpiar el texto
def limpiar_texto(texto):
    if isinstance(texto, str):  # Verificar si el texto es una cadena
        texto = texto.lower()
        texto = re.sub(r"[^a-zA-Z0-9áéíóúñü\s]", "", texto)
        return texto
    return ""  # Retornar una cadena vacía si no es texto

In [ ]:
# Crear la columna clean_text
data['clean_text'] = data['text'].apply(limpiar_texto

In [ ]:

# Verificar las primeras filas para confirmar que la limpieza funcionó
print(data[['text', 'clean_text']].head())

In [ ]:
# Generar datos simulados para los equipos
teams = ["Real Madrid", "Barcelona", "Atlético Madrid", "Valencia", "Villarreal", "Sevilla"]
data_historica = []

In [ ]:
# Datos reales: posiciones históricas de los últimos 5 años
posiciones_historicas = {
    "Real Madrid": [1, 2, 1, 2, 1],
    "Barcelona": [2, 1, 3, 1, 2],
    "Atlético Madrid": [3, 3, 2, 3, 3],
    "Valencia": [7, 9, 13, 4, 6],
    "Villarreal": [5, 7, 5, 5, 4],
    "Sevilla": [4, 4, 4, 6, 5]
}

In [ ]:
# Datos reales: capacidad de los estadios (en miles de espectadores)
capacidad_estadios = {
    "Real Madrid": 81,
    "Barcelona": 99,
    "Atlético Madrid": 68,
    "Valencia": 49,
    "Villarreal": 23,
    "Sevilla": 43
}

In [ ]:
# Generar datos simulados y calcular predicciones basadas en posiciones históricas
for team in teams:
    prob_victoria = round(random.uniform(0.5, 1.0), 2)  # Probabilidad entre 0.5 y 1.0
    goles_favor = random.randint(0, 5)  # Goles promedio por partido
    goles_contra = random.randint(0, 5)

    # Predicción basada en posiciones históricas
    posiciones = posiciones_historicas.get(team, [10, 10, 10, 10, 10])
    prediccion_posicion = round(sum(posiciones) / len(posiciones))

    # Capacidad del estadio
    capacidad_estadio = capacidad_estadios.get(team, 30)  # Valor predeterminado si falta

    data_historica.append({
        "team": team,
        "prob_victoria": prob_victoria,
        "goles_favor": goles_favor,
        "goles_contra": goles_contra,
        "posicion_tabla": prediccion_posicion,
        "capacidad_estadio": capacidad_estadio
    })

data_historica_df = pd.DataFrame(data_historica)
print("Datos históricos y simulados con capacidad del estadio:")
print(data_historica_df)

In [ ]:
# Preparar datos históricos para integrarlos con las noticias
# Alinear los índices entre los datos históricos y las noticias
if len(data) > len(data_historica_df):
    data_historica_df = data_historica_df.sample(len(data), replace=True, random_state=42).reset_index(drop=True)

data['historical_data'] = data_historica_df.apply(
    lambda x: [x["prob_victoria"], x["goles_favor"], x["goles_contra"], x["posicion_tabla"], x["capacidad_estadio"]], axis=1
)
data["historical_data"] = data["historical_data"].apply(lambda x: ','.join(map(str, x)))

In [ ]:
# Tokenización del texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['clean_text'])
tokenized_texts = tokenizer.texts_to_sequences(data['clean_text'])

In [ ]:
# Creación de secuencias y padding
seq_length = 50
input_sequences = []
historical_features = []
targets = []

for i, seq in enumerate(tokenized_texts):
    for j in range(seq_length, len(seq)):
        input_sequences.append(seq[j-seq_length:j])
        targets.append(seq[j])
        historical_features.append([float(val) for val in data['historical_data'].iloc[i].split(',')])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length))
targets = np.array(targets)
historical_features = np.array(historical_features)

In [ ]:
# Ajustar datos históricos para que coincidan con las secuencias
historical_features_adjusted = []

In [ ]:
# Alinear correctamente los datos históricos con las secuencias
for i in range(len(input_sequences)):
    index = i % len(historical_features)  # Usar módulo para repetir los datos históricos
    historical_features_adjusted.append(historical_features[index])

In [ ]:
# Convertir a array de numpy
historical_features_adjusted = np.array(historical_features_adjusted)

In [ ]:
# Verificar dimensiones
print(f"Dimensiones ajustadas de historical_features: {historical_features_adjusted.shape}")
print(f"Dimensiones de input_sequences: {input_sequences.shape}")
print(f"Dimensiones de targets: {targets.shape}")

In [ ]:
# Definir el tamaño del vocabulario
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Definir las entradas del modelo
text_input = Input(shape=(seq_length,), name="text_input")
historical_input = Input(shape=(len(historical_features_adjusted[0]),), name="historical_input")

In [ ]:
# Embedding y LSTM para el texto
embedding = Embedding(vocab_size, 128, input_length=seq_length)(text_input)
lstm = LSTM(256, return_sequences=False)(embedding)

In [ ]:
# Concatenar la salida de LSTM con los datos históricos
concat = Concatenate()([lstm, historical_input])

In [ ]:
# Capas densas
dense = Dense(128, activation='relu')(concat)
output = Dense(vocab_size, activation='softmax')(dense)

In [ ]:
# Construir el modelo
model = tf.keras.models.Model(inputs=[text_input, historical_input], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

In [ ]:
# Parámetros de entrenamiento
batch_size = 64
epochs = 30

# Entrenar el modelo
history = model.fit(
    [input_sequences, historical_features_adjusted],  # Entradas
    targets,                                          # Salidas
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2
)

In [ ]:
# Definir una función para generar texto
def generate_text(seed_text, historical_data, next_words=200):
    seed_text += f" con una probabilidad de victoria del {historical_data[0]*100:.1f}%, "
    seed_text += f"goles a favor promedio de {historical_data[1]}, y una capacidad del estadio de {historical_data[4]} mil espectadores."

    for _ in range(next_words):
        tokenized_seed = tokenizer.texts_to_sequences([seed_text])[0]
        tokenized_seed = pad_sequences([tokenized_seed], maxlen=seq_length, padding='pre')
        prediction = model.predict([tokenized_seed, np.array([historical_data])], verbose=0)
        predicted_word = tokenizer.index_word.get(np.argmax(prediction), "")
        seed_text += " " + predicted_word
    return seed_text

# Generar texto de ejemplo
seed_text = "the victory the game"
historical_data_example = [0.85, 3, 1, 2, 81]  # Ejemplo de datos históricos
generated_news = generate_text(seed_text, historical_data_example)
print("Noticia generada:")
print(generated_news)